<a href="https://colab.research.google.com/github/siren7075/hackathon2024/blob/main/thresholds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get Anomalies for SPOT, need to sort by date and group by currency

In [31]:
import pandas as pd
from scipy import stats

# Load the CSV data
df = pd.read_csv('fx_rates_spot_forward_MID_3mo_fromUSD.csv')

# Filter the DataFrame for fxType "SPOT"
spot_df = df[df['fxType'] == 'SPOT']

# Calculate percentage changes in exchange rates
spot_df['percentage_change'] = spot_df['spotRate'].pct_change() * 100

# Calculate mean and standard deviation of percentage changes
mean_change = spot_df['percentage_change'].mean()
std_dev_change = spot_df['percentage_change'].std()

# Define thresholds using Z-score method
k = 2  # Adjust k for strictness
lower_threshold = mean_change - k * std_dev_change
upper_threshold = mean_change + k * std_dev_change

# Identify anomalies based on the thresholds
spot_df['anomaly'] = ((spot_df['percentage_change'] < lower_threshold) |
                       (spot_df['percentage_change'] > upper_threshold))

# Display the results
print("Dynamic Thresholds:")
print(f"Lower Threshold: {lower_threshold:.2f}%")
print(f"Upper Threshold: {upper_threshold:.2f}%")
print("\nAnomalies based on dynamic thresholds:")
print(spot_df[spot_df['anomaly']][['effectiveDate', 'toCurrencyCode', 'spotRate', 'percentage_change']])

Dynamic Thresholds:
Lower Threshold: -470378.54%
Upper Threshold: 543323.22%

Anomalies based on dynamic thresholds:
      effectiveDate toCurrencyCode  spotRate  percentage_change
370      2024-09-11            IDR   15400.0       2.153282e+06
812      2024-09-12            IDR   15430.0       2.162183e+06
1296     2024-09-13            IDR   15400.0       2.169144e+06
1716     2024-09-16            IDR   15400.0       2.175843e+06
2585     2024-09-18            IDR   15335.0       2.167962e+06
...             ...            ...       ...                ...
26946    2024-12-05            IDR   15860.0       2.130057e+06
27345    2024-12-06            IDR   15850.0       2.127842e+06
27816    2024-12-09            IDR   15865.0       2.130887e+06
28211    2024-12-10            IDR   15865.0       2.119385e+06
28650    2024-12-11            IDR   15915.0       2.121927e+06

[66 rows x 4 columns]


<ipython-input-31-5d0c2da4e342>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spot_df['percentage_change'] = spot_df['spotRate'].pct_change() * 100
<ipython-input-31-5d0c2da4e342>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spot_df['anomaly'] = ((spot_df['percentage_change'] < lower_threshold) |


In [23]:
import pandas as pd

df = pd.read_csv('fx_rates_spot_forward_MID_3mo_fromUSD.csv')

# example fxType "SPOT" and toCurrencyCode "JPY"
filtered_df = df[(df['fxType'] == 'SPOT') & (df['toCurrencyCode'] == 'JPY')]

mean_spot_rate = filtered_df['spotRate'].mean()
std_spot_rate = filtered_df['spotRate'].std()

# Define dynamic thresholds (e.g., mean ± 2 standard deviations)
lower_threshold = mean_spot_rate - 2 * std_spot_rate
upper_threshold = mean_spot_rate + 2 * std_spot_rate

anomalies = filtered_df[(filtered_df['spotRate'] < lower_threshold) | (filtered_df['spotRate'] > upper_threshold)]

print("Dynamic Thresholds:")
print(f"Lower Threshold: {lower_threshold}")
print(f"Upper Threshold: {upper_threshold}")
print("\nAnomalies based on dynamic thresholds:")
print(anomalies[['effectiveDate', 'spotRate']])

Dynamic Thresholds:
Lower Threshold: 140.92620444027506
Upper Threshold: 158.27425010517948

Anomalies based on dynamic thresholds:
     effectiveDate  spotRate
1239    2024-09-13    140.42
1708    2024-09-16    140.51


Using ZScore

In [26]:
import pandas as pd
from scipy import stats

df = pd.read_csv('fx_rates_spot_forward_MID_3mo_fromUSD.csv')

# example fxType "SPOT" and toCurrencyCode "JPY"
filtered_df = df[(df['fxType'] == 'SPOT') & (df['toCurrencyCode'] == 'JPY')]

filtered_df['z_score'] = stats.zscore(filtered_df['spotRate'])
mean_z_score = filtered_df['z_score'].mean()
std_z_score = filtered_df['z_score'].std()

# For example, using mean ± 2 * std deviation of Z-scores
dynamic_upper_threshold = mean_z_score + 2 * std_z_score
dynamic_lower_threshold = mean_z_score - 2 * std_z_score

outliers = filtered_df[(filtered_df['z_score'] > dynamic_upper_threshold) | (filtered_df['z_score'] < dynamic_lower_threshold)]

print("Dynamic Thresholds:")
print(f"Upper Threshold (Z-score): {dynamic_upper_threshold}")
print(f"Lower Threshold (Z-score): {dynamic_lower_threshold}")
print("\nOutliers based on dynamic Z-score thresholds:")
print(outliers[['effectiveDate', 'spotRate', 'z_score']])

Dynamic Thresholds:
Upper Threshold (Z-score): 2.0153258946231167
Lower Threshold (Z-score): -2.015325894623114

Outliers based on dynamic Z-score thresholds:
     effectiveDate  spotRate   z_score
1239    2024-09-13    140.42 -2.132938
1708    2024-09-16    140.51 -2.112027


<ipython-input-26-329e9edb4153>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['z_score'] = stats.zscore(filtered_df['spotRate'])
